<a href="https://colab.research.google.com/github/p-rakesh/Python-scripts/blob/main/Technical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Provides ways to work with large multidimensional arrays
import numpy as np
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

import yfinance as yf
from plotly.subplots import make_subplots

!pip install ta

from ta.trend import MACD
from ta.momentum import StochasticOscillator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df=yf.download(tickers='AMD',start='2022-06-19',end='2023-05-21',interval='1d')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-06-21,84.169998,85.809998,82.599998,83.790001,83.790001,87780600
2022-06-22,84.400002,86.379997,83.300003,83.750000,83.750000,86634700
2022-06-23,84.320000,84.410004,80.230003,82.430000,82.430000,100614600
2022-06-24,83.559998,87.529999,83.080002,87.080002,87.080002,88553900
2022-06-27,87.360001,88.220001,85.250000,86.160004,86.160004,74663500
...,...,...,...,...,...,...
2023-05-15,95.199997,97.430000,93.449997,97.400002,97.400002,51749200
2023-05-16,97.389999,103.279999,97.309998,101.480003,101.480003,90622900
2023-05-17,101.790001,104.139999,100.050003,103.750000,103.750000,75240900


In [ ]:
df['MA_20']=df['Adj Close'].ewm(span=20,adjust= False).mean()
df['MA_50']=df['Adj Close'].ewm(span=50,adjust= False).mean()
df

,Open,High,Low,Close,Adj Close,Volume,MA_20,MA_50
Date,,,,,,,,
2022-06-21,84.169998,85.809998,82.599998,83.790001,83.790001,87780600,83.790001,83.790001
2022-06-22,84.400002,86.379997,83.300003,83.750000,83.750000,86634700,83.786191,83.788432
2022-06-23,84.320000,84.410004,80.230003,82.430000,82.430000,100614600,83.657030,83.735160
2022-06-24,83.559998,87.529999,83.080002,87.080002,87.080002,88553900,83.983028,83.866331
2022-06-27,87.360001,88.220001,85.250000,86.160004,86.160004,74663500,84.190359,83.956279
...,...,...,...,...,...,...,...,...
2023-05-15,95.199997,97.430000,93.449997,97.400002,97.400002,51749200,92.229371,89.746756
2023-05-16,97.389999,103.279999,97.309998,101.480003,101.480003,90622900,93.110383,90.206883
2023-05-17,101.790001,104.139999,100.050003,103.750000,103.750000,75240900,94.123680,90.737986


In [ ]:
fig=go.Figure()

candle= go.Candlestick(x=df.index, open=df['Open'],high=df['High'],low=df['Low'],close=df['Close'], name='Candlestick')
ema20 = go.Scatter(x=df.index, y=df['MA_20'],
                   line=dict(color='green', width=1), name="EMA20")

ema50 = go.Scatter(x=df.index, y=df['MA_50'],
                  line=dict(color='orange', width=1), name="EMA50")

# Add plots to the figure
fig.add_trace(candle)
fig.add_trace(ema20)
fig.add_trace(ema50)
fig.update_layout(title="AMD")

# Get rid of empty dates on the weekend
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
fig.show()

In [ ]:
def plot_macd_stoch(ticker, time_period, time_int):
    # Download stock data
    stock_df = yf.download(tickers=ticker, period=time_period, interval=time_int)

    # Define candlestick and moving average lines
    candle = go.Candlestick(x=stock_df.index, open=stock_df['Open'],
                       high=stock_df['High'], low=stock_df['Low'],
                       close=stock_df['Close'])
    # A period of 12 and 26 are commonly used
    stock_df['MA12'] = stock_df['Adj Close'].ewm(span=12, adjust=False).mean()
    stock_df['MA26'] = stock_df['Adj Close'].ewm(span=26, adjust=False).mean()

    # Calculate the MACD
    # A signal line uses a period of 9 and when it crosses the other moving
    # average it is a sign to buy or sell
    macd = MACD(close=stock_df['Close'],
             window_slow=26,
             window_fast=12,
             window_sign=9)

    # A Stochastic (stuh ka stuhk) Oscillator is also plotted because it
    # gives us a signal of an upcoming trend reversal
    # Values range from 0 to 100 and values over 80 are considered to be
    # overbought while those under 20 are considered oversold

    # We calculate normally over a 14 day period
    # We are smoothing price data
    sto_os = StochasticOscillator(high=stock_df['High'],
                             close=stock_df['Close'],
                             low=stock_df['Low'],
                             window=14,
                             smooth_window=3)

    # Make 4 subplots on separate rows
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

    fig.add_trace(candle, row=1, col=1)

    # Add the histogram, MACD and Signal lines

    # The MacD line is a calculation made by subtracting the 26 period
    # EMA from the 12 period EMA
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)

    # The signal is calculated by taking the average of the last 9 values
    # of the MACD line (The signal line is a slower more smoothed out version of
    # the MACD line)
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)

    # The histogram shows the difference between the MACD and signal line
    # When the MACD line is below the signal that is a negative value and vice versa
    fig.add_trace(go.Bar(x=stock_df.index,
                     y=macd.macd_diff()
                    ), row=2, col=1)

    # The MacD line is a calculation made by subtracting the 26 period
    # EMA from the 12 period EMA
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)

    # The signal is calculated by taking the average of the last 9 values
    # of the MACD line (The signal line is a slower more smoothed out version of
    # the MACD line)
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)

    # Plot Stochastics
    # This is the faster of the 2 lines called the K line
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch(),
                         line=dict(color='blue', width=2)
                        ), row=3, col=1)
    # This line is slower and is known as the D line and it is an average of the K line
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch_signal(),
                         line=dict(color='orange', width=2)
                        ), row=3, col=1)

    # Draw 20 and 80 lines
    fig.add_hline(y=20, line_width=1, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=80, line_width=1, line_dash="dash", line_color="green", row=3, col=1)

    # Add volume
    fig.add_trace(go.Bar(x=stock_df.index,
                     y=stock_df['Volume']
                    ), row=4, col=1)

    # Update titles
    fig.update_layout(title=ticker)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="StoOs", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)

    fig.update_layout(height=900, width=1200,
                  showlegend=False,
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(bounds=["sat", "mon"]), dict(bounds=[16, 9.5], pattern="hour")])

    fig.show()

In [ ]:
plot_macd_stoch("PG", "5d", "5m")

[*********************100%***********************]  1 of 1 completed
